In [1]:
! pip install transformers

     ---------------------------------------- 7.4/7.4 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 1.6 MB/s eta 0:00:00
     -------------------------------------- 263.9/263.9 kB 1.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import streamlit as st

import torch
from torch.utils.data import DataLoader ,Dataset
from transformers import AutoTokenizer,BertForQuestionAnswering,AutoModel

In [3]:
from transformers import AutoTokenizer,BertForQuestionAnswering,AutoModel
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Emam\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
from transformers import DataCollatorWithPadding

In [5]:
torch.set_default_device('cpu')

In [6]:
from transformers import BertTokenizer, BertModel


In [7]:
class bert_compare(torch.nn.Module):
    def __init__ (self):
        super(bert_compare,self).__init__()
        self.bert=BertModel.from_pretrained("bert-base-uncased")
        
        self.Linear=torch.nn.Linear(768,30 )
        self.elu=torch.nn.ELU()
        self.Linear2=torch.nn.Linear(280 ,1 )
        self.cnn1=torch.nn.Conv1d(768,256,kernel_size=2)
        self.cnn2=torch.nn.Conv1d(256,10,kernel_size=2)

        self.relu=torch.nn.ReLU()
    def forward(self,x):
        x=self.bert(**x).last_hidden_state
        x=x.permute(0,2,1)
        x=self.cnn1(x)
        x=self.relu(x)
        x=self.cnn2(x)
        x=torch.nn.Flatten()(x)
        x=self.Linear2(x)
        return x

In [8]:
model=bert_compare()
optim=torch.optim.AdamW(model.parameters(),lr=5e-5)
loss=torch.nn.BCEWithLogitsLoss()


In [9]:
def tok(x,y):
        out=tokenizer(x,y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        out={key:value for key,value in out.items()}
        return out
h=tok('my name is mohamed','what is your name')    
model(h)

tensor([[0.0010]], grad_fn=<AddmmBackward0>)

In [ ]:
! pip install tqdm

In [10]:
from  tqdm import tqdm

In [20]:
model.train()

bert_compare(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [12]:
model.bert.train()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [15]:
word=['my name is mohamed ', "How do I read and find my YouTube comments?" ,"How can I see all my Youtube comments?","How can Internet speed be increased by hacking through DNS?","What is the step by step guide to invest in share market in india?","where is capital of egypt?",'when did you born ','what is your name',"what is capital of egypt",'how old are you']

In [16]:
def tok(x,y):
        out=tokenizer(x,y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        out={key:value for key,value in out.items()}
        return out
for i in range(9):
        r=torch.randint(len(word),size=(1,))
        r2=torch.randint(len(word),size=(1,))
        h=tok(word[r],word[r2])    
        e=model(h)
        ans= 'the same' if  int(torch.sigmoid( e)>=.5) else 'not the same'
        print (f'{word[r]} is {ans} {word[r2]}'  )

how old are you is not the same what is capital of egypt
what is your name is not the same when did you born 
What is the step by step guide to invest in share market in india? is not the same when did you born 
what is your name is not the same when did you born 
How can Internet speed be increased by hacking through DNS? is not the same What is the step by step guide to invest in share market in india?
How can I see all my Youtube comments? is not the same what is your name
How can I see all my Youtube comments? is not the same when did you born 
How do I read and find my YouTube comments? is not the same when did you born 
How can I see all my Youtube comments? is not the same when did you born 
